In [1]:
import cv2
import numpy as np

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from srgan2 import SRGAN

Using TensorFlow backend.


In [4]:
fname = 'UBMk30rjy0o.mp4'
path = 'data/val/' + fname

In [5]:
cap = cv2.VideoCapture(path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 動画の画面横幅
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 動画の画面縦幅
fcnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 総フレーム数
fps = cap.get(cv2.CAP_PROP_FPS)

for _ in range(1000):
    ret, frame = cap.read()
low_frame = cv2.resize(frame , (int(w/2), int(h/2)))

In [6]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_dir_val = 'data/val/low/'
tgt_dir_val = 'data/val/high/'


test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (540, 960, 3)
tgt_shape = (1080, 1920, 3)

batch_size = 2

In [7]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)

/home/keras/srgan2.py:114: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
/home/keras/srgan2.py:141: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [12]:
from keras.models import Model, load_model

In [13]:
m = load_model('models/20200512_generator.h5')

In [14]:
m.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 270, 480, 3)  0                                            
__________________________________________________________________________________________________
G_Head_conv0 (Conv2D)           (None, 270, 480, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
G_Head_act0 (PReLU)             (None, 270, 480, 32) 4147200     G_Head_conv0[0][0]               
__________________________________________________________________________________________________
G_Body_0_conv0 (Conv2D)         (None, 270, 480, 32) 9248        G_Head_act0[0][0]                
____________________________________________________________________________________________

In [10]:
model = srgan.generator
model.load_weights('models/20200512_generator.h5')

ValueError: Shapes (540, 960, 32) and (270, 480, 32) are incompatible

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 540, 960, 3)  0                                            
__________________________________________________________________________________________________
G_Head_conv0 (Conv2D)           (None, 540, 960, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
G_Head_act0 (PReLU)             (None, 540, 960, 32) 16588800    G_Head_conv0[0][0]               
__________________________________________________________________________________________________
G_Body_0_conv0 (Conv2D)         (None, 540, 960, 32) 9248        G_Head_act0[0][0]                
____________________________________________________________________________________________

In [18]:
low_frame.shape

(540, 960, 3)

In [19]:
img = (low_frame - 127.5 ) / 127.5
img_gen = model.predict(img[np.newaxis, :, :, :])

img_int = ((img_gen * 127.5) + 127.5).astype('u1').reshape((1080, 1920, 3))
cv2.imwrite('test.jpg', img_int)

True

In [20]:
cv2.imwrite('test_low.jpg', low_frame)

True